In [1]:
from src.reference import Reference
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.tools import MMSeqs
import requests
import re
from tqdm import tqdm
import os

%load_ext autoreload 
%autoreload 2


In [6]:
reference = Reference('/home/prichter/Downloads/gbffs/GCF_000006825.1_genomic.gbff', load_contigs=True)
reference.get_contig_ids()

['NC_002663']

In [9]:
reference.get_nt_seq(contig_id='NC_002663', start=2003130 - 1, stop=2003540) # complement(1268411..1268656) 2003130..2003540

'ATGCGTAATGTGAAAAAAGCATTGTCAAGATCTGTTATTGCTAGTAGTATTCAAGGTACTACTAAAACCAACGGCGGACACCGCAGCCGTCACTCAAGCGGTATTTTTTTACCTCAAATTTACCTTTCAAAAGGAAATGGGGCATATCCCCACACCTTTCAAAAAGCCGAATTTGTGGCAAGGGCGACACTCCGAAATAAGGCGCATTTGCGCACGAATAAGGGGGGCTTTCATCCGTTGGTGGTAGTAGTTGAACCCTTGTCACGCCTACTAAGCGTTGGCAAGTTTTTACAAAACAAACCAACGGAAAAAATCAAAATGAAAACCAGTCAAAAATCCACCGCACTTTTAGCGGTTCGTTCACGCACACCTATTGTTTTTGCTCCTGCTATGGAGGTGACTTATGCGTAA'

In [13]:
arr = [[np.arange(10), np.arange(5), np.arange(2)]]
arr = [[np.arange(30), np.arange(8), np.arange(4)]]
for x in zip(*arr):
    print(np.concatenate(x))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[0 1 2 3 4 5 6 7]
[0 1 2 3]
